In [ ]:
# Test the new embedding tool

from embedding_tool import KnowledgeBaseEmbedder

# Create embedder instance
embedder = KnowledgeBaseEmbedder()

# Create embeddings and get statistics
result = embedder.create_embeddings()

# Display results
print(f"Status: {result['status']}")
print(f"Message: {result['message']}")
print(f"Documents loaded: {result['documents_loaded']}")
print(f"Chunks created: {result['chunks_created']}")
print(f"Vectors stored: {result['vectors_stored']}")
print(f"Vector dimensions: {result['vector_dimensions']}")
print(f"Document types found: {', '.join(result['doc_types'])}")

In [ ]:
# Test retrieval from the new vectorstore

vectorstore = embedder.get_vectorstore()

if vectorstore:
    # Test a sample query
    query = "software development experience"
    results = vectorstore.similarity_search(query, k=3)
    
    print(f"Query: '{query}'")
    print(f"Found {len(results)} relevant documents:")
    print()
    
    for i, doc in enumerate(results, 1):
        print(f"Result {i}:")
        print(f"  Doc type: {doc.metadata.get('doc_type', 'unknown')}")
        print(f"  Source: {doc.metadata.get('source', 'unknown')}")
        print(f"  Content preview: {doc.page_content[:200]}...")
        print()
else:
    print("No vectorstore found")